In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%set_env CUDA_LAUNCH_BLOCKING=1

env: CUDA_LAUNCH_BLOCKING=1


In [3]:
import sys
import collections
import cv2
import os
import math
import random
import pickle
import copy
import numpy as np
sys.path.append("C:\\Users\\user\\Documents\\Workspace\\MeronymNet-PyTorch\\src")

In [4]:
import torch
import torch.optim as optim
from torch_geometric.data import Data
from torch_geometric.utils import dense_to_sparse
from torch_geometric.data import DataLoader
import torch.utils.data as data_utils
from torch.utils.tensorboard import SummaryWriter

In [25]:
from losses import BoxVAE_losses as loss
from evaluation import metrics
from utils import plot_utils
from utils import data_utils as data_loading
from components.DenseAutoencoder import DenseAutoencoder
from components.DenseAutoencoder import Decoder
from components.TwoStageAutoEncoder import TwoStageAutoEncoder

In [6]:
import matplotlib.pyplot as plt

In [7]:
# from mask_generation import masked_sketch

In [22]:
latent_dims = 64
num_nodes = 16
bbx_size = 4
num_classes = 7
label_shape = 1
nb_epochs = 500
klw = loss.frange_cycle_linear(nb_epochs)
learning_rate = 0.00005
hidden1 = 32
hidden2 = 16
hidden3 = 128
dense_hidden1=16
dense_hidden2=32
adaptive_margin = True
fine_tune_box = False
output_log = False
area_encoding = False
run_prefix = "two_stage_obj_parts_conditioning_weighted_IOU"
variational=False

In [ ]:
batch_train_loader, batch_val_loader = data_loading.load_data(obj_data_postfix = '_obj_boundary',
                                                              part_data_post_fix = '_scaled',
                                                              file_postfix = '_combined',
                                                              seed=345,
                                                              batch_size=batch_size)

In [20]:
import gc
gc.collect()

90

In [21]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [23]:
reconstruction_loss_arr = []
kl_loss_arr = []
bbox_loss_arr = []
adj_loss_arr = []
node_loss_arr = []

reconstruction_loss_val_arr = []
kl_loss_val_arr = []
bbox_loss_val_arr = []
adj_loss_val_arr = []
node_loss_val_arr = []

vae = TwoStageAutoEncoder(latent_dims,
                          num_nodes,
                          bbx_size,
                          num_classes,
                          label_shape,
                          hidden1,
                          hidden2,
                          hidden3,
                          dense_hidden1,
                          dense_hidden2,
                          adaptive_margin,
                          output_log,
                          area_encoding
                        )
vae.to(device)
optimizer = optim.Adam(vae.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[200,400], gamma=0.75)
model_path = ('D:/meronym_data/model/'+run_prefix+'/Obj-Box-'
                        +str(learning_rate)
                        +'-batch-'+str(batch_size)
                        +'-h1-'+str(hidden1)
                        +'-h2-'+str(hidden2)
                        +'-h3-'+str(hidden3)+'-test')
summary_path = ('D:/meronym_data/runs/'+run_prefix+'/Obj-Box-'
                        +str(learning_rate)
                        +'-batch-'+str(batch_size)
                        +'-h1-'+str(hidden1)
                        +'-h2-'+str(hidden2)
                        +'-h3-'+str(hidden3)+'-test')
if not os.path.exists(model_path):
    os.makedirs(model_path)
writer = SummaryWriter(summary_path)
icoef = 0

for epoch in range(nb_epochs):
    
    batch_loss = torch.tensor([0.0])
    batch_kl_loss = torch.tensor([0.0])
    batch_bbox_loss = torch.tensor([0.0])
    batch_obj_bbox_loss = torch.tensor([0.0])
    batch_node_loss = torch.tensor([0.0])
    IOU_weight_delta = torch.tensor([(1+epoch)/nb_epochs])
    images = []
    
    vae.train()
    i=0
    for train_data in batch_train_loader:
        
        node_data_true = train_data.x
        label_true = node_data_true[:,:1]
        y_true = train_data.y
        class_true = y_true[:, :num_classes]
        X_obj_true = y_true[:, num_classes:]
        adj_true = train_data.edge_index
        batch = train_data.batch
#         node_data_true  = torch.reshape(node_data_true, (int(batch.shape[0]/num_nodes), num_nodes, node_data_true.shape[-1]))
        
        class_true  = torch.flatten(class_true)
        

        for param in vae.parameters():
            param.grad=None
        
        output = vae(adj_true, node_data_true, X_obj_true, label_true , class_true, False)
        node_data_pred, X_obj_pred, label_pred, z_mean, z_logvar, margin = output
        obj_bbox_loss = loss.obj_bbox_loss(pred_box=X_obj_pred, true_box=X_obj_true)
        kl_loss = loss.kl_loss(z_mean, z_logvar)
        bbox_loss = loss.weighted_bbox_loss(pred_box=node_data_pred, true_box=node_data_true[:,1:], weight=IOU_weight_delta, margin=margin)
        node_loss = loss.node_loss(label_pred,label_true)
        
        kl_weight = klw[icoef]
        
        if kl_weight>0:
            reconstruction_loss = kl_loss*kl_weight + (obj_bbox_loss + bbox_loss + node_loss)*num_nodes
        else:
            reconstruction_loss = (obj_bbox_loss + bbox_loss + node_loss)*num_nodes
            
        reconstruction_loss.backward()
        
        optimizer.step()
        
        i+=1
      
        batch_loss += reconstruction_loss
        batch_kl_loss += kl_loss
        batch_bbox_loss += bbox_loss
        batch_obj_bbox_loss += obj_bbox_loss
        batch_node_loss += node_loss
    
        if i%200==0:
            print(i)
            global_step = epoch*len(batch_train_loader)+i
            
            writer.add_scalar("Loss/train/reconstruction_loss", batch_loss.item()/(i+1), global_step)
            writer.add_scalar("Loss/train/kl_loss", batch_kl_loss.item()/(i+1), global_step)
            writer.add_scalar("Loss/train/bbox_loss", batch_bbox_loss.item()/(i+1), global_step)
            writer.add_scalar("Loss/train/obj_bbox_loss", batch_obj_bbox_loss.item()/(i+1), global_step)
            writer.add_scalar("Loss/train/node_loss", batch_node_loss.item()/(i+1), global_step)
            
            
#     scheduler.step()
    global_step = epoch*len(batch_train_loader)+i
    image_shape = [num_nodes, bbx_size]

    image = plot_utils.plot_bbx(np.reshape((node_data_true[:num_nodes,1:5]*label_true[:num_nodes]).detach().to("cpu").numpy(),
                                image_shape)).astype(float)/255
    writer.add_image('train/images/input', image, global_step, dataformats='HWC')
    image = plot_utils.plot_bbx((node_data_pred[0]*label_true[:num_nodes]).detach().to("cpu").numpy()).astype(float)/255
    writer.add_image('train/images/generated', image, global_step, dataformats='HWC')
    
    reconstruction_loss_arr.append(batch_loss.detach().item()/(i+1))
    kl_loss_arr.append(batch_kl_loss.detach().item()/(i+1))
    bbox_loss_arr.append(batch_bbox_loss.detach().item()/(i+1))
    node_loss_arr.append(batch_node_loss.detach().item()/(i+1))
    
    print('[%d, %5d] loss: %.3f' %
              (epoch + 1, i + 1, batch_loss/(i+1) ))
    
    
    batch_loss = torch.tensor([0.0])
    batch_kl_loss = torch.tensor([0.0])
    batch_bbox_loss = torch.tensor([0.0])
    batch_obj_bbox_loss = torch.tensor([0.0])
    batch_node_loss = torch.tensor([0.0])
    images = []
    vae.eval()
    for i, val_data in enumerate(batch_val_loader, 0):
        
        node_data_true = val_data.x
        label_true = node_data_true[:,:1]
        y_true = val_data.y
        class_true = torch.flatten(y_true[:, :num_classes])
        X_obj_true = y_true[:, num_classes:]
        adj_true = val_data.edge_index
        batch = val_data.batch
#         node_data_true  = torch.reshape(node_data_true, (int(batch.shape[0]/num_nodes), num_nodes, node_data_true.shape[-1]))
        
        class_true  = torch.flatten(class_true)
        
        output = vae(adj_true, node_data_true, X_obj_true, label_true , class_true, False)
        node_data_pred, X_obj_pred, label_pred, z_mean, z_logvar, margin = output
        obj_bbox_loss = loss.obj_bbox_loss(pred_box=X_obj_pred, true_box=X_obj_true)
        kl_loss = loss.kl_loss(z_mean, z_logvar)
        bbox_loss = loss.weighted_bbox_loss(pred_box=node_data_pred, true_box=node_data_true[:,1:], weight=IOU_weight_delta, margin=margin)
        node_loss = loss.node_loss(label_pred,label_true)
        
        if kl_weight>0:
            reconstruction_loss = kl_loss*kl_weight + (obj_bbox_loss + bbox_loss + node_loss)*num_nodes
        else:
            reconstruction_loss = (obj_bbox_loss + bbox_loss + node_loss)*num_nodes
            
        batch_loss += reconstruction_loss
        batch_kl_loss += kl_loss
        batch_bbox_loss += bbox_loss
        batch_obj_bbox_loss += obj_bbox_loss
        batch_node_loss += node_loss
    
    image = plot_utils.plot_bbx(np.reshape((node_data_true[:num_nodes,1:5]*label_true[:num_nodes]).detach().to("cpu").numpy(),
                                image_shape)).astype(float)/255
    writer.add_image('val/images/input', image, global_step, dataformats='HWC')
    image = plot_utils.plot_bbx((node_data_pred[0]*label_true[:num_nodes]).detach().to("cpu").numpy()).astype(float)/255
    writer.add_image('val/images/generated', image, global_step, dataformats='HWC')

    reconstruction_loss_val_arr.append(batch_loss.detach().item()/(i+1))
    kl_loss_val_arr.append(batch_kl_loss.detach().item()/(i+1))
    bbox_loss_val_arr.append(batch_bbox_loss.detach().item()/(i+1))
    node_loss_val_arr.append(batch_node_loss.detach().item()/(i+1))
    
    writer.add_scalar("Loss/val/reconstruction_loss", batch_loss.detach()/(i+1), global_step)
    writer.add_scalar("Loss/val/kl_loss", batch_kl_loss.item()/(i+1), global_step)
    writer.add_scalar("Loss/val/bbox_loss", batch_bbox_loss.item()/(i+1), global_step)
    writer.add_scalar("Loss/val/obj_bbox_loss", batch_obj_bbox_loss.item()/(i+1), global_step)
    writer.add_scalar("Loss/val/node_loss", batch_node_loss.item()/(i+1), global_step)
       
    if epoch%50 == 0:
        torch.save(vae.state_dict(), model_path + '/model_weights.pth')
        
#     if kl_loss_arr[-1]>0.5 and abs(bbox_loss_arr[-1] - bbox_loss_val_arr[-1]) < 0.02 and bbox_loss_arr[-1]<1.3 and epoch>60:
#         icoef = icoef + 1  

torch.save(vae.state_dict(),model_path + '/model_weights.pth')

for i in range(min(100,int(len(node_data_true)/num_nodes))):    
    image = plot_utils.plot_bbx(np.reshape((node_data_true[num_nodes*i:num_nodes*(i+1),1:5]).detach().to("cpu").numpy(),
                                    image_shape)).astype(float)/255
    writer.add_image('result/images/'+str(i)+'-input', image, global_step, dataformats='HWC')
    image = plot_utils.plot_bbx((node_data_pred[i]*label_true[num_nodes*i:num_nodes*(i+1)]).detach().to("cpu").numpy()).astype(float)/255
    writer.add_image('result/images/'+str(i)+'-generated', image, global_step, dataformats='HWC')
    
writer.flush()
writer.close()
print('Finished Training')

200
[1,   399] loss: 131.486
200
[2,   399] loss: 97.483
200
[3,   399] loss: 93.980
200
[4,   399] loss: 92.448
200
[5,   399] loss: 91.596
200
[6,   399] loss: 91.072
200
[7,   399] loss: 90.541
200
[8,   399] loss: 89.878
200
[9,   399] loss: 89.182
200
[10,   399] loss: 88.585
200
[11,   399] loss: 88.106
200
[12,   399] loss: 87.608
200
[13,   399] loss: 87.198
200
[14,   399] loss: 86.779
200
[15,   399] loss: 86.224
200
[16,   399] loss: 85.415
200
[17,   399] loss: 84.069
200
[18,   399] loss: 82.147
200
[19,   399] loss: 79.914
200
[20,   399] loss: 77.858
200
[21,   399] loss: 75.887
200
[22,   399] loss: 73.113
200
[23,   399] loss: 68.603
200
[24,   399] loss: 64.029
200
[25,   399] loss: 61.106
200
[26,   399] loss: 59.270
200
[27,   399] loss: 57.921
200
[28,   399] loss: 56.881
200
[29,   399] loss: 56.036
200
[30,   399] loss: 55.264
200
[31,   399] loss: 54.558
200
[32,   399] loss: 53.797
200
[33,   399] loss: 52.824
200
[34,   399] loss: 51.275
200
[35,   399] loss: 

200
[279,   399] loss: 9.079
200
[280,   399] loss: 9.048
200
[281,   399] loss: 9.017
200
[282,   399] loss: 8.986
200
[283,   399] loss: 8.956
200
[284,   399] loss: 8.927
200
[285,   399] loss: 8.897
200
[286,   399] loss: 8.866
200
[287,   399] loss: 8.836
200
[288,   399] loss: 8.807
200
[289,   399] loss: 8.778
200
[290,   399] loss: 8.749
200
[291,   399] loss: 8.720
200
[292,   399] loss: 8.690
200
[293,   399] loss: 8.662
200
[294,   399] loss: 8.634
200
[295,   399] loss: 8.606
200
[296,   399] loss: 8.577
200
[297,   399] loss: 8.549
200
[298,   399] loss: 8.522
200
[299,   399] loss: 8.493
200
[300,   399] loss: 8.465
200
[301,   399] loss: 8.438
200
[302,   399] loss: 8.411
200
[303,   399] loss: 8.385
200
[304,   399] loss: 8.357
200
[305,   399] loss: 8.330
200
[306,   399] loss: 8.304
200
[307,   399] loss: 8.278
200
[308,   399] loss: 8.251
200
[309,   399] loss: 8.224
200
[310,   399] loss: 8.198
200
[311,   399] loss: 8.172
200
[312,   399] loss: 8.146
200
[313,   39

KeyboardInterrupt: 

In [28]:
#testing loop
model_path = ('D:/meronym_data/model/'+run_prefix+'/Obj-Box-'
                        +str(learning_rate)
                        +'-batch-'+str(batch_size)
                        +'-h1-'+str(hidden1)
                        +'-h2-'+str(hidden2)
                        +'-h3-'+str(hidden3)+'-test')
summary_path = ('D:/meronym_data/runs/'+run_prefix+'/Obj-Box-'
                        +str(learning_rate)
                        +'-batch-'+str(batch_size)
                        +'-h1-'+str(hidden1)
                        +'-h2-'+str(hidden2)
                        +'-h3-'+str(hidden3)+'-test')


write_tensorboard = False
if write_tensorboard:
    writer = SummaryWriter(summary_path)

vae = TwoStageAutoEncoder(latent_dims,
                          num_nodes,
                          bbx_size,
                          num_classes,
                          label_shape,
                          hidden1,
                          hidden2,
                          hidden3,
                          dense_hidden1,
                          dense_hidden2,
                          adaptive_margin,
                          output_log,
                          area_encoding
                        )

vae.load_state_dict(torch.load(model_path+ '/model_weights.pth'))

# decoder = vae.decoder
image_shape = [num_nodes, bbx_size]
global_step = 250000
class_dict = {0:'cow', 1:'sheep', 2:'bird', 3:'person', 4:'cat', 5:'dog', 6:'horse'}
res_dfs = []
for i, val_data in enumerate(batch_val_loader, 0):
    
    val_data.cuda()
    node_data_true = val_data.x
    label_true = node_data_true[:,:1]
    y_true = val_data.y
    class_true = y_true[:, :num_classes]
    X_obj_true = y_true[:, num_classes:]
    adj_true = val_data.edge_index
    class_true  = torch.flatten(class_true)
    
    output = vae(adj_true, node_data_true, X_obj_true, label_true , class_true)
    node_data_pred_test, X_obj_pred_test, label_pred, z_mean, z_logvar, margin = output
    
    res_dfs.append(metrics.get_metrics(node_data_true, X_obj_true, node_data_pred_test,
                               X_obj_pred_test,
                               label_true,
                               class_true,
                               num_nodes,
                               num_classes))
    
    if write_tensorboard:
        
        for j in range(int(len(node_data_true)/num_nodes)):
            
            image = plot_bbx(node_data_true[j].detach().to("cpu").numpy().astype(float))/255
            pred_image = plot_bbx(node_data_pred_test[j].detach().to("cpu").numpy()).astype(float)/255

            writer.add_image('test_result/images/'+str(j)+'-input/', image, global_step, dataformats='HWC')  
            writer.add_image('test_result/images/'+str(j)+'-generated/', pred_image, global_step, dataformats='HWC')
            

result = pd.concat(res_dfs)
result['obj_class'] = np.where(result['obj_class'].isna(), 0, result['obj_class'])
result['obj_class'] = result['obj_class'].astype('int')
result['obj_class'].replace(class_dict, inplace=True)
result.where(result['part_labels']!=0, np.NaN, inplace=True)
result['part_labels'] = np.where(result['part_labels'].isna(), 0, result['part_labels'])
result['part_labels'] = result['part_labels'].astype('int')
result['id'] = np.repeat(np.array(list(range(int(len(result)/num_nodes)))), 16)

if write_tensorboard:
    writer.flush()
    writer.close()

bird_labels = {'head':1 , 'torso':2, 'neck':3, 'lwing':4, 'rwing':5, 'lleg':6, 'lfoot':7, 'rleg':8, 'rfoot':9, 'tail':10}
cat_labels = {'head':1, 'torso':2, 'neck':3, 'lfleg':4, 'lfpa':5, 'rfleg':6, 'rfpa':7, 'lbleg':8, 'lbpa':9, 'rbleg':10, 'rbpa':11, 'tail':12}
cow_labels = {'head':1,'lhorn':2, 'rhorn':3, 'torso':4, 'neck':5, 'lfuleg':6, 'lflleg':7, 'rfuleg':8, 'rflleg':9, 'lbuleg':10, 'lblleg':11, 'rbuleg':12, 'rblleg':13, 'tail':14}
dog_labels = {'head':1,'torso':2, 'neck':3, 'lfleg':4, 'lfpa':5, 'rfleg':6, 'rfpa':7, 'lbleg':8, 'lbpa':9, 'rbleg':10, 'rbpa':11, 'tail':12, 'muzzle':13}
horse_labels = {'head':1,'lfho':2, 'rfho':3, 'torso':4, 'neck':5, 'lfuleg':6, 'lflleg':7, 'rfuleg':8, 'rflleg':9, 'lbuleg':10, 'lblleg':11, 'rbuleg':12, 'rblleg':13, 'tail':14, 'lbho':15, 'rbho':16}
person_labels = {'head':1, 'torso':2, 'neck': 3, 'llarm': 4, 'luarm': 5, 'lhand': 6, 'rlarm':7, 'ruarm':8, 'rhand': 9, 'llleg': 10, 'luleg':11, 'lfoot':12, 'rlleg':13, 'ruleg':14, 'rfoot':15}
sheep_labels = cow_labels
part_labels_combined_parts = {'bird': bird_labels, 'cat': cat_labels, 'cow': cow_labels, 'dog': dog_labels, 'sheep': sheep_labels, 'horse':horse_labels,'person':person_labels}

for k, v in part_labels_combined_parts.items():
    new_map = {}
    for pk, pv in v.items():
        new_map[pv]=pk
    part_labels_combined_parts[k] = new_map
    
for k, v in part_labels_combined_parts.items():
    result.loc[result['obj_class']==k, ['part_labels']] = result.loc[result['obj_class']==k,['part_labels']].replace(v).copy()

result.to_csv(model_path+'/raw_metrics.csv')
res_obj_level = result.groupby(['obj_class', 'id']).mean().reset_index()
res_obj_level.groupby(['obj_class']).mean().reset_index()[['obj_class', 'IOU', 'MSE']].to_csv(model_path+'/obj_level_metrics.csv')
result.groupby(['obj_class', 'part_labels']).mean().reset_index()[['obj_class', 'part_labels',  'IOU', 'MSE']].to_csv(model_path+'/part_level_metrics.csv')

In [29]:
model_path = ('D:/meronym_data/model/'+run_prefix+'/Obj-Box-'
                        +str(learning_rate)
                        +'-batch-'+str(batch_size)
                        +'-h1-'+str(hidden1)
                        +'-h2-'+str(hidden2)
                        +'-h3-'+str(hidden3)+'-test')
summary_path = ('D:/meronym_data/runs/'+run_prefix+'/Obj-Box-'
                        +str(learning_rate)
                        +'-batch-'+str(batch_size)
                        +'-h1-'+str(hidden1)
                        +'-h2-'+str(hidden2)
                        +'-h3-'+str(hidden3)+'-test')

result = pd.read_csv(model_path+'/raw_metrics.csv')
result.groupby(['obj_class', 'part_labels']).mean().reset_index()[['obj_class', 'part_labels',  'IOU', 'MSE']]

,obj_class,part_labels,IOU,MSE
0,bird,head,0.588108,0.000508
1,bird,lfoot,0.205085,0.001241
2,bird,lleg,0.292030,0.000658
3,bird,lwing,0.376934,0.001889
4,bird,neck,0.422315,0.000640
...,...,...,...,...
81,sheep,lhorn,0.206568,0.001257
82,sheep,neck,0.641831,0.000292
83,sheep,rhorn,0.271497,0.000685
84,sheep,tail,0.284374,0.000417


In [30]:
res_obj_level = result.groupby(['obj_class', 'id']).mean().reset_index()
res_obj_level.groupby(['obj_class']).mean().reset_index()[['obj_class', 'IOU', 'MSE']]

,obj_class,IOU,MSE
0,bird,0.514180,0.000707
1,cat,0.565921,0.001979
2,cow,0.524824,0.000638
3,dog,0.447616,0.001275
4,horse,0.416515,0.000894
5,person,0.484889,0.001070
6,sheep,0.650987,0.000341


In [31]:
res_obj_level[['IOU', 'MSE']].mean()

IOU    0.508807
MSE    0.001078
dtype: float64

In [32]:
result.groupby(['obj_class', 'part_labels']).mean().reset_index()[['obj_class', 'part_labels',  'IOU', 'MSE']]

,obj_class,part_labels,IOU,MSE
0,bird,head,0.588108,0.000508
1,bird,lfoot,0.205085,0.001241
2,bird,lleg,0.292030,0.000658
3,bird,lwing,0.376934,0.001889
4,bird,neck,0.422315,0.000640
...,...,...,...,...
81,sheep,lhorn,0.206568,0.001257
82,sheep,neck,0.641831,0.000292
83,sheep,rhorn,0.271497,0.000685
84,sheep,tail,0.284374,0.000417


In [35]:
X_obj_true[:,1]+X_obj_true[:,3]

tensor([1.0001, 1.0015, 1.0023, 0.9985, 0.9999, 0.9977, 0.9985, 1.0015, 1.0000,
        0.9985, 0.9985, 0.9999, 0.9977, 1.0000, 1.0015, 0.9977, 1.0000, 1.0001,
        0.9985, 1.0008, 1.0023, 0.9985, 0.9985, 1.0008, 0.9985, 0.9999, 1.0000,
        0.9985, 1.0023, 0.9999, 0.9985, 0.9985, 1.0015, 0.9999, 1.0001, 0.9985,
        1.0015, 1.0015, 1.0023, 1.0008, 0.9992, 1.0015, 0.9992, 1.0001, 1.0008,
        1.0001, 1.0023, 0.9985, 0.9985, 1.0008, 1.0015, 0.9992, 1.0015, 0.9977,
        1.0008, 0.9985, 0.9985, 1.0008, 0.9977, 1.0023, 0.9985, 1.0008, 1.0000,
        1.0015])

In [38]:
X_obj_pred[:, 1]+X_obj_pred[:, 3]

tensor([0.9994, 0.9999, 1.0002, 0.9988, 0.9995, 1.0005, 0.9996, 1.0015, 0.9999,
        1.0010, 0.9998, 0.9997, 1.0012, 0.9994, 0.9994, 0.9994, 1.0012, 0.9997,
        0.9999, 1.0016, 1.0012, 0.9993, 1.0001, 1.0000, 0.9994, 0.9991, 0.9997,
        1.0008, 1.0010, 0.9999, 1.0012, 0.9997, 0.9992, 1.0007, 0.9992, 0.9996,
        1.0005, 0.9998, 1.0012, 0.9992, 1.0011, 1.0005, 1.0004, 1.0009, 1.0001,
        0.9998, 1.0008, 0.9992, 0.9996, 0.9997, 0.9996, 0.9989, 1.0005, 1.0008,
        1.0016, 1.0011, 1.0004, 1.0013, 1.0010, 1.0001, 0.9990, 0.9997, 0.9999,
        1.0005, 0.9996, 1.0010, 0.9999, 1.0005, 1.0010, 1.0012, 0.9988, 0.9998,
        1.0007, 1.0007, 1.0000, 0.9994, 1.0008, 0.9999, 0.9998, 1.0005, 1.0017,
        0.9997, 1.0006, 0.9996, 1.0005, 0.9993, 1.0013, 0.9993, 0.9990, 0.9998,
        1.0001, 1.0010, 1.0008, 0.9990, 1.0011, 1.0005, 0.9995, 1.0011, 1.0003,
        1.0008, 0.9993, 1.0001, 1.0002, 0.9991, 0.9996, 0.9988, 0.9995, 1.0008,
        1.0008, 1.0002, 0.9998, 1.0008, 

In [39]:
X_obj_true[:10], X_obj_pred[:10]

(tensor([[0.2777, 0.3223, 0.7224, 0.6778],
         [0.2676, 0.2863, 0.7325, 0.7152],
         [0.3704, 0.4354, 0.6319, 0.5669],
         [0.4136, 0.3833, 0.5848, 0.6152],
         [0.3847, 0.4228, 0.6138, 0.5771],
         [0.2605, 0.2682, 0.7372, 0.7295],
         [0.3720, 0.4372, 0.6265, 0.5613],
         [0.3036, 0.2906, 0.6965, 0.7109],
         [0.3212, 0.1864, 0.6773, 0.8136],
         [0.3909, 0.2833, 0.6076, 0.7152]]),
 tensor([[0.2502, 0.2611, 0.7488, 0.7383],
         [0.3786, 0.4173, 0.6210, 0.5826],
         [0.3537, 0.4144, 0.6462, 0.5858],
         [0.2920, 0.3404, 0.7065, 0.6584],
         [0.2866, 0.1400, 0.7124, 0.8596],
         [0.3467, 0.3365, 0.6531, 0.6639],
         [0.3505, 0.4214, 0.6485, 0.5782],
         [0.2099, 0.3002, 0.7910, 0.7013],
         [0.2450, 0.2542, 0.7543, 0.7457],
         [0.2705, 0.3613, 0.7302, 0.6397]], grad_fn=<SliceBackward>))

In [ ]:
outfile = 'D:/meronym_data/generate_boxes.npy'
with open(outfile, 'wb') as pickle_file:
    pred_boxes = np.concatenate(pred_boxes)
    pickle.dump(pred_boxes, pickle_file)
outfile = 'D:/meronym_data/generate_boxesobj_class.npy'
with open(outfile, 'wb') as pickle_file:
    pickle.dump(classes,pickle_file)